In [ ]:
import os
from joblib import load
from tensorflow.keras.models import load_model
from keras.models import load_model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import itertools


# Helper functions for data preprocessing and feature extraction
from methods.preprocessing import *
from methods.image_enhancement import *

# Machine Learning models and utilities
from methods.ml import *

# CNN models and utilities
from methods.cnn import *

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.model_selection import cross_val_score


In [ ]:
!unzip "data/fer-2013-dataset.zip"

In [ ]:
# Read data in folder
test_dir = "test"
for dirpath,dirname,filenames in os.walk(test_dir):
    print(f"There are {len(dirname)} directories and {len(filenames)} images in {dirpath[22:]}")

In [ ]:
data_gen = create_data_gen()
validation_generator = data_gen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical"
)

validation_vgg_generator = data_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical"
)

In [ ]:
cnn_model = load_model("cnn_model.h5")
cnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
results = cnn_model.evaluate(validation_generator)
print("Loss:", results[0], "Accuracy:", results[1])

In [ ]:
cnn_model = load_model("cnn_vgg_model.h5")
cnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
results = cnn_model.evaluate(validation_vgg_generator)
print("Loss:", results[0], "Accuracy:", results[1])

In [ ]:
validation_vgg_generator.reset()
cnn_model = load_model("cnn_improved_vgg_model.h5")
cnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
predictions = cnn_model.predict(validation_vgg_generator)
results = cnn_model.evaluate(validation_vgg_generator)
print("Loss:", results[0], "Accuracy:", results[1])
predicted_class_indices = np.argmax(predictions, axis=1)
true_class_indices = validation_vgg_generator.classes
cm = confusion_matrix(true_class_indices, predicted_class_indices)

In [ ]:
class_labels = list(validation_vgg_generator.class_indices.keys())  

# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plotting non-normalized confusion matrix
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cm, classes=class_labels)
plt.show()


In [ ]:
print(classification_report(true_class_indices, predicted_class_indices, target_names=class_labels))
